In [1]:
import numpy as np
from scipy.special import expit  # Logistic function

import pandas as pd
UnprocessedDF = pd.read_csv('UnprocessedDF.csv')


Going to make a few presumptions here. Firstly that the amount of work done at week 48 is representative of their general work and that we are classifying full-time work at 30 hours and above and part time work as less than 30.

In [156]:
# Going to make a few presumptions here. Firstly that the amount of work done at week 48 is representative of their general work and that
# we are classifying full-time work at 30 hours and above and part time work as less than 30. Non-responses are given a 0

EmploymentCols = [col for col in UnprocessedDF.columns if "EMP_HOURS" in col]

def recode_work(value):
    if value >= 30:
        return 2
    elif value >= 0 and value < 30:
        return 1
    elif value == -4:
        return 0 
    return value

UnprocessedDF[EmploymentCols] = UnprocessedDF[EmploymentCols].applymap(recode_work)


for col in EmploymentCols:
    # Remove rows where the column value is -3
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -3]



We are going to combine divorced and seperated here and to remove entries where there was no data given for relationship data. We are also going to remove instances of widowed as those are very far and in-between.

In [157]:
MarrCols = [col for col in UnprocessedDF.columns if "MAR_STATUS" in col]

# Combining divorced and seperated 

def recode_marriage(value):
    if value == 4:
        return 3
    return value

UnprocessedDF[MarrCols] = UnprocessedDF[MarrCols].applymap(recode_marriage)


for col in MarrCols:
    # Remove rows where the column value is -3
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -3]


# Recode so that a non-response for the first 13 years is simply coded as non-married as all 14 and under are not interviewed.

MarrColsFirst3Years = ['MAR_STATUS_1997.12','MAR_STATUS_1998.12']

def recode_marriageNA(value):
    if value == -4:
        return 0
    return value

UnprocessedDF[MarrColsFirst3Years] = UnprocessedDF[MarrColsFirst3Years].applymap(recode_marriageNA)

MarrCols = [value for value in MarrCols if value not in MarrColsFirst3Years]

for col in MarrCols:
    #Removing Widowed
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -1]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -2]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -3]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -4]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -5]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != 5]

In [158]:
#UnprocessedDF.to_csv('Semiprocessed.csv')

Code to rename the Sexually Active Column so that it is in two Formats. We have the columns Sexually_Active and Sexually_Active_B. 

If the interviewee has responded "Yes" to being sexually active at the intiial interview or being sexually active in a year then we determine that they have bencome sexually active.  

In [159]:
#DATING STUFF. We must first code it so.

#The logic is a bit strange due to the fact that the logic is coded in a weird way. If anyone has been interviewed in the past, then they will be skipped for the question (HAS ever had sex)
# However, they will still be asked every year or so about whether any sexual activity has occured since the last interview.

def create_rename_dict():
    rename_dict = {}
    
    # The first year without any periods
    rename_dict["YSAQ-299"] = "SexuallyActive_1997"
    rename_dict["YSAQ-299B"] = "SexuallyActive_1998B"
    rename_dict["YSAQ2-299"] = "SexuallyActive_2002"
    rename_dict["YSAQ2-299B"] = "SexuallyActive_2002B"

#First iteration should always be hardcoded and the rest from the loop
    
    # Loop through years from 1998 to 2009
    for year in range(1998, 2002):  # Years from 1998 to 2009
        index = year - 1997  # Calculate the index for .1, .2, etc.
        rename_dict[f"YSAQ-299.{index}"] = f"SexuallyActive_{year}"

#Somewhat complicated because of the DLI aspect. It works so that if anyone is polled once they they go to the next variable

    for year in range(1999, 2002):  # Years from 1998 to 2009
        index = year - 1998  # Calculate the index for .1, .2, etc.
        rename_dict[f"YSAQ-299B.{index}"] = f"SexuallyActive_{year}B"

    for year in range(2003, 2006):  # Years from 1998 to 2009
        index = year - 2002  # Calculate the index for .1, .2, etc.
        rename_dict[f"YSAQ2-299.{index}"] = f"SexuallyActive_{year}"


    for year in range(2003, 2012):  # Years from 1998 to 2009
        index = year - 2002  # Calculate the index for .1, .2, etc.
        rename_dict[f"YSAQ2-299B.{index}"] = f"SexuallyActive_{year}B"

    rename_dict["YSAQ2-299.4"] = "SexuallyActive_2013"
    rename_dict["YSAQ2-299.5"] = "SexuallyActive_2015"


    return rename_dict

# Generate the renaming dictionary
RenameSexuallyActive = create_rename_dict()

In [160]:
UnprocessedDF.rename(columns=RenameSexuallyActive, inplace=True)

In [161]:
UnprocessedDF = UnprocessedDF[sorted(UnprocessedDF.columns)]

In [162]:
#Function to Compare the two values for every year, if they say yes FOR EITHER column then they are determined to be sexually active

def comparesexual(row, col1, col2):
    #Check if sexually active ever
    if (row[col1] == 1 or row[col2] == 1):
        return 1
    # Because clearly not sexually ACTIVE ever:
    elif row[col1] == 0:
        return 0 
    else:
        return row[col2]
    

In [163]:
# List of years to loop through
years = list(range(1998, 2006))

for year in years:
    col1 = f'SexuallyActive_{year}'
    col2 = f'SexuallyActive_{year}B'
    new_col_name = f'FinalState_{year}'

    UnprocessedDF[new_col_name] = UnprocessedDF.apply(lambda row: comparesexual(row, col1, col2), axis=1)

In [164]:
remaindersexualactivity = {
    'SexuallyActive_1997': 'FinalState_1997',
    'SexuallyActive_2006B': 'FinalState_2006',
    'SexuallyActive_2007B': 'FinalState_2007',
    'SexuallyActive_2008B': 'FinalState_2008',
    'SexuallyActive_2009B': 'FinalState_2009',
    'SexuallyActive_2010B': 'FinalState_2010',
    'SexuallyActive_2011B': 'FinalState_2011',
    'SexuallyActive_2013': 'FinalState_2013',
    'SexuallyActive_2015': 'FinalState_2015'

}
UnprocessedDF.rename(columns=remaindersexualactivity, inplace=True)

UnprocessedDF = UnprocessedDF[sorted(UnprocessedDF.columns)]

In [165]:
#Subjects under the age of 14 are not asked about sexual activity therefore we will assume that the valid non-response are simply nos. They however, will get the interview at age 14 so it makes no major difference.

def recode_UA97(row):
    if row['FinalState_1997'] == -4 and (row['KEY!AGEDOL'] == 12 or row['KEY!AGEDOL'] == 13):
        return 0  # Recode col1 to 99 if both are 1
    else:
        return row['FinalState_1997']  # Keep original value otherwise

#Think there was a sampling gap here. 

def recode_UA98(row):
    if row['FinalState_1998'] == -4 and (row['KEY!AGEDOL'] == 12 or row['KEY!AGEDOL'] == 13):
        return 0  # Recode col1 to 99 if both are 1
    else:
        return row['FinalState_1998']  # Keep original value otherwise

# Apply the function to recode col1
UnprocessedDF['FinalState_1997'] = UnprocessedDF.apply(recode_UA97, axis=1)

UnprocessedDF['FinalState_1998'] = UnprocessedDF.apply(recode_UA98, axis=1)

Sadly, the even years are not polled after the year 2011 and there is no data for after 2015 so we make the assumption that everyone after the year 2015 retains the same value.

In [166]:
# Extra Category for Missing Data. Presume that the 

UnprocessedDF['FinalState_2012'] = UnprocessedDF['FinalState_2011']
UnprocessedDF['FinalState_2014'] = UnprocessedDF['FinalState_2013']
UnprocessedDF['FinalState_2016'] = UnprocessedDF['FinalState_2015']
UnprocessedDF['FinalState_2016'] = UnprocessedDF['FinalState_2015']
UnprocessedDF['FinalState_2017'] = UnprocessedDF['FinalState_2015']
UnprocessedDF['FinalState_2018'] = UnprocessedDF['FinalState_2015']
UnprocessedDF['FinalState_2019'] = UnprocessedDF['FinalState_2015']

In [167]:
#Sorting the Values Year by Year

SexualActivitycols = sorted([col for col in UnprocessedDF.columns if "FinalState_" in col])

Relationshipcols = sorted([col for col in UnprocessedDF.columns if "MAR_STATUS_" in col])


#UnprocessedDF[MarrCols] = UnprocessedDF[MarrCols].applymap(recode_marriage)
#Note: Rs without any biological children are assigned a valid skip (-4).


This next block of code does two things.

1. Because it is possible for individuals to be sexually active but not having been sexually active within the last year and because of skipped surveys, we loop through the dataframe so that if an individual has ever been sexually active, the 1 value carries on.

2. Also because of the possbility of a missed survey, we decided to encode it so that marriage implies being sexually active

In [168]:
def comparemarriage(row, marr, pastsexualhist):
    if (row[marr] == 0):
        return row[pastsexualhist]
    elif (row[marr] != 0):
        return 1

#First pass on values where sexually active individuals remain sexually active in further years


final_state_columns = sorted([col for col in UnprocessedDF.columns if col.startswith("FinalState_")], key=lambda x: int(x.split('_')[1]))

# Loop through each row
for index, row in UnprocessedDF.iterrows():
    # Initialize a flag to track if a previous column has a value of 1
    found_one = False
    
    # Loop through the identified columns
    for col in final_state_columns:
        if found_one:
            # If a previous column had a value of 1, set current column to 1
            UnprocessedDF.at[index, col] = 1
        elif row[col] == 1:
            # If current column is 1, set the flag
            found_one = True



for i,(a,b) in enumerate(zip(Relationshipcols,SexualActivitycols)):
    # Apply logic so that the married individuals are sexually active
    UnprocessedDF[b] = UnprocessedDF.apply(lambda row: comparemarriage(row, a, b), axis=1)



In [169]:
# Running the first instance of the code one more time in the case that the marriage variable changes things

final_state_columns = sorted([col for col in UnprocessedDF.columns if col.startswith("FinalState_")], key=lambda x: int(x.split('_')[1]))

# Loop through each row
for index, row in UnprocessedDF.iterrows():
    # Initialize a flag to track if a previous column has a value of 1
    found_one = False
    
    # Loop through the identified columns
    for col in final_state_columns:
        if found_one:
            # If a previous column had a value of 1, set current column to 1
            UnprocessedDF.at[index, col] = 1
        elif row[col] == 1:
            # If current column is 1, set the flag
            found_one = True


for col in SexualActivitycols:
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -1]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -2]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -3]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -4]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -5]



Renaming the First Column of the Child Category so that it can be easily looped. The variable CV_BIO_CHILD_HH means having a child in the same HH and CV_BIO_CHILD_NR means having a child living in a different household.

In [170]:
updatechildcount= {
    'CV_BIO_CHILD_HH': 'CV_BIO_CHILD_HH.0',
    'CV_BIO_CHILD_NR': 'CV_BIO_CHILD_NR.0',
    'CV_BIO_CHILD_HH_U18': 'CV_BIO_CHILD_HH.18',
    'CV_BIO_CHILD_NR_U18': 'CV_BIO_CHILD_NR.18',
}

UnprocessedDF.rename(columns=updatechildcount, inplace=True)

In [171]:
#Since subjects without kids are not polled we assume that they are not parents. Elsewise we code it so they if they have a child they are a parent.

def checkbirths(row, col1, col2):
    # Return 1 if either value is 0 or above, else return 0
    if row[col1] > 0 or row[col2] > 0:
        return 1
    elif row[col1] == -4 or row[col2] == -4:
        return 0
    else:
        return row[col1]

In [172]:
suffix = range(0, 20)

ChildinHouse = sorted([col for col in UnprocessedDF.columns if "CV_BIO_CHILD_HH" in col])
ChildNotinHouse = sorted([col for col in UnprocessedDF.columns if "CV_BIO_CHILD_NR" in col])

listofyears = [1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2013,2015,2017,2019]

for i,(a,b) in enumerate(zip(ChildinHouse,ChildNotinHouse)):
    # Apply logic so that the married individuals are sexually active
    NewColumnName = "ChildProduced_" + str(listofyears[i])
    UnprocessedDF[NewColumnName] = UnprocessedDF.apply(lambda row: checkbirths(row, a, b), axis=1)


# Once again no data after 2015. 
    
UnprocessedDF['ChildProduced_2012'] = UnprocessedDF['ChildProduced_2011']
UnprocessedDF['ChildProduced_2014'] = UnprocessedDF['ChildProduced_2013']
UnprocessedDF['ChildProduced_2016'] = UnprocessedDF['ChildProduced_2015']
UnprocessedDF['ChildProduced_2018'] = UnprocessedDF['ChildProduced_2017']


In [173]:
#Recode Non-Answers to be a 0 since these students have never been interviewed and I don't want to drop the DS

def recode_Child97(row):
    if row['ChildProduced_1997'] == -5 and (row['KEY!AGEDOL'] == 12 or row['KEY!AGEDOL'] == 13 or row['KEY!AGEDOL'] == 14):
        return 0  # Recode col1 to 99 if both are 1
    else:
        return row['ChildProduced_1997']  # Keep original value otherwise

#Think there was a sampling gap here. 

def recode_Child98(row):
    if row['ChildProduced_1998'] == -5 and (row['KEY!AGEDOL'] == 12 or row['KEY!AGEDOL'] == 13 or row['KEY!AGEDOL'] == 14):
        return 0  # Recode col1 to 99 if both are 1
    else:
        return row['ChildProduced_1998']  # Keep original value otherwise

# Apply the function to recode col1
UnprocessedDF['ChildProduced_1997'] = UnprocessedDF.apply(recode_Child97, axis=1)

UnprocessedDF['ChildProduced_1998'] = UnprocessedDF.apply(recode_Child98, axis=1)

Logic to essentially apply that if someone has ever had a child they will be a parent in subsequent years

In [174]:
childcolumns = sorted([col for col in UnprocessedDF.columns if col.startswith("ChildProduced_")], key=lambda x: int(x.split('_')[1]))

# Loop through each row
for index, row in UnprocessedDF.iterrows():
    # Initialize a flag to track if a previous column has a value of 1
    found_one = False
    
    # Loop through the identified columns
    for col in childcolumns:
        if found_one:
            # If a previous column had a value of 1, set current column to 1
            UnprocessedDF.at[index, col] = 1
        elif row[col] == 1:
            # If current column is 1, set the flag
            found_one = True

Likewise, if they are not a parent in a given year, then we assume that they are not a parent in previous years

In [175]:
childcolumnsreverse = sorted([col for col in UnprocessedDF.columns if col.startswith("ChildProduced_")], key=lambda x: int(x.split('_')[1]),reverse=True)

# Loop through each row
for index, row in UnprocessedDF.iterrows():
    # Initialize a flag to track if a previous column has a value of 1
    found_one = False
    
    # Loop through the identified columns
    for col in childcolumnsreverse:
        if found_one:
            # If a previous column had a value of 0, set current column to 0
            UnprocessedDF.at[index, col] = 0
        elif row[col] == 0:
            # If current column is 1, set the flag
            found_one = True

In [176]:
UnprocessedDF = UnprocessedDF[sorted(UnprocessedDF.columns)]

In [177]:
for col in childcolumns:
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -1]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -2]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -3]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -4]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -5]

Because we are missing education data post 2011, but have a smaller dataset that tells us whether someone is enrolled. We then map the values for continuing education. However, if someone is not contiuing education we assume that their last education is final.

In [178]:
listofyears = [1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2013,2015,2017,2019]


educationcolumn = []
educationcolumn.append(['CV_ENROLLSTAT'])
educationcolumn.append(sorted([col for col in UnprocessedDF.columns if "CV_ENROLLSTAT_EDT" in col]))
educationcolumn.append(sorted([col for col in UnprocessedDF.columns if "CV_ENROLLSTAT." in col]))
educationcolumn.append(sorted([col for col in UnprocessedDF.columns if "SCH_COLLEGE_STATUS" in col]))


educationcolumn = [item for sublist in educationcolumn for item in sublist]


In [179]:



fulllistofyears = [1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

for i,a in enumerate(educationcolumn):
    NewColumnName = "EducationStat_" + str(fulllistofyears[i])
    UnprocessedDF[NewColumnName] = UnprocessedDF[a]
    

In [180]:
#If we are studying then we recode it so that it's mapped. However, if we are not studying then we take the last good value from before the new poll format

def recode_collegestat(row, oldyr, newyr):
    if row[newyr] == 2:
        return 9  # Recode to 0 if conditions are met
    elif row[newyr] == 3:
        return 10
    elif row[newyr] == 4:
        return 11
    elif row[newyr] == 1:
        return 99
    else:
        return row[oldyr]  # Keep original value otherwise

# Apply the function to recode education according to last good value
UnprocessedDF['EducationStat_2012'] = UnprocessedDF.apply(lambda row: recode_collegestat(row, 'EducationStat_2011', 'EducationStat_2012'), axis=1)
UnprocessedDF['EducationStat_2013'] = UnprocessedDF.apply(lambda row: recode_collegestat(row, 'EducationStat_2012', 'EducationStat_2013'), axis=1)
UnprocessedDF['EducationStat_2014'] = UnprocessedDF.apply(lambda row: recode_collegestat(row, 'EducationStat_2013', 'EducationStat_2014'), axis=1)
UnprocessedDF['EducationStat_2015'] = UnprocessedDF.apply(lambda row: recode_collegestat(row, 'EducationStat_2014', 'EducationStat_2015'), axis=1)
UnprocessedDF['EducationStat_2016'] = UnprocessedDF.apply(lambda row: recode_collegestat(row, 'EducationStat_2015', 'EducationStat_2016'), axis=1)
UnprocessedDF['EducationStat_2017'] = UnprocessedDF.apply(lambda row: recode_collegestat(row, 'EducationStat_2016', 'EducationStat_2017'), axis=1)
UnprocessedDF['EducationStat_2018'] = UnprocessedDF.apply(lambda row: recode_collegestat(row, 'EducationStat_2017', 'EducationStat_2018'), axis=1)
UnprocessedDF['EducationStat_2019'] = UnprocessedDF.apply(lambda row: recode_collegestat(row, 'EducationStat_2018', 'EducationStat_2019'), axis=1)



In [181]:
EducationColumns = sorted([col for col in UnprocessedDF.columns if "EducationStat_" in col])
EducationColumnsReverse = EducationColumns[::-1]

We further replace empty values with the last valid value.

In [182]:
#Replaces Invalid values with the Value Last Given

ValuesToReplace = [-4,-5,-3,-2,-1]

for index, row in UnprocessedDF.iterrows():
    recent_outside_value = None
    
    # Iterate over each column in the sorted list
    for col in EducationColumns:
        if row[col] in ValuesToReplace:
            # Replace with the first value outside the list if found
            if recent_outside_value is not None:
                UnprocessedDF.at[index, col] = recent_outside_value
        else:
            # Record the first value outside the list
                recent_outside_value = row[col]
    


#However, if one is in education, we then assume that they graduate after and thereby are promoted to the next good value.

ValuesToReplace = [99]
NonStableValues = [9,10,11]




In [183]:
#Remove

for index, row in UnprocessedDF.iterrows():
    recent_outside_value = None
    
    # Iterate over each column in the sorted list
    for col in EducationColumns:
        if row[col] in ValuesToReplace:
            # Replace with the first value outside the list if found
            if recent_outside_value == 9:
                UnprocessedDF.at[index, col] = 5
            elif recent_outside_value == 10:
                UnprocessedDF.at[index, col] = 6
            elif recent_outside_value == 11:
                UnprocessedDF.at[index, col] = 7
            else:
                UnprocessedDF.at[index, col] = recent_outside_value
        else:
            # Record the first value outside the list
                recent_outside_value = row[col]

In [184]:
for col in EducationColumns:
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -1]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -2]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -3]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -4]
    UnprocessedDF = UnprocessedDF[UnprocessedDF[col] != -5]

In [185]:
#Remove the ".48" and the'.12' so that we can loop.

#Remove all columns

UnprocessedDF.columns = UnprocessedDF.columns.str.replace(r'\.48$', '', regex=True)

UnprocessedDF.columns = UnprocessedDF.columns.str.replace(r'\.12$', '', regex=True)
UnprocessedDF['HasSibling'] = UnprocessedDF['YOUTH_SIBID01.01']

UnprocessedDF.loc[UnprocessedDF['HasSibling'] > 0, 'HasSibling'] = 1
UnprocessedDF.loc[UnprocessedDF['HasSibling'] == -4, 'HasSibling'] = 0


# Remove people with No Parents living in the Household

UnprocessedDF = UnprocessedDF[UnprocessedDF['YOUTH_PARENTGUAR.01'] == 1]

# Single Parent or Double Parent HH = 0 for Single 1 for Double
UnprocessedDF['SingleorDoubleParent'] = UnprocessedDF['YOUTH_NONR1INHH.01']

#Loop through a LIST OF COLUMNS and then find what we should keep

ListOfColumnstoKeep = ['ChildProduced','EMP_Hours','FinalState','KEY!AGEDOL','KEY!SEX','MAR_STATUS','EducationStat','HasSibling','SingleorDoubleParent','EMP_HOURS']

columns_to_keep = [col for col in UnprocessedDF.columns if any(value in col for value in ListOfColumnstoKeep)]


UnprocessedDF = UnprocessedDF[columns_to_keep]

In [186]:
#Create a new column to allow for the replacement of the marriage column

# So originally it would be 0 for Single, 1 for Cohabilitating, 2 for Marriage, 3 for Seperated.

# We will change that so that  0 for Single and Not Sexually Active, 1 for Single and Sexually Active, 2 for Single and SA with a child, 3 for Cohabitating, 4 for Cohabilitating with a child
# 5 for Married, 6 for Married with a Child, 7 for Divorced, 8 for Divorced with a Child


def createfinalmarr(row, marriage, sexuallyactive,kids):
    #Check if sexually active ever
    if (row[marriage] == 0 and row[sexuallyactive] == 0):
        return 0
    elif (row[marriage] == 0 and row[sexuallyactive] == 1 and row[kids] == 0):
        return 1
    elif (row[marriage] == 0 and row[sexuallyactive] == 1 and row[kids] == 1):
        return 2
    elif (row[marriage] == 1 and row[kids] == 0):
        return 3
    elif (row[marriage] == 1 and row[kids] == 1):
        return 4
    elif (row[marriage] == 2 and row[kids] == 0):
        return 5
    elif (row[marriage] == 2 and row[kids] == 1):
        return 6
    elif (row[marriage] == 3 and row[kids] == 0):
        return 7
    elif (row[marriage] == 3 and row[kids] == 1):
        return 8
    else:
        return 999


SexualActivitycols = sorted([col for col in UnprocessedDF.columns if "FinalState_" in col])
Relationshipcols = sorted([col for col in UnprocessedDF.columns if "MAR_STATUS_" in col])
Childproduced = sorted([col for col in UnprocessedDF.columns if "ChildProduced_" in col])

fulllistofyears = [1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]




for i,(a,b,c) in enumerate(zip(Relationshipcols,SexualActivitycols,Childproduced)):
    # Apply logic so that the married individuals are sexually active
    NewColumnName = "FinalRelationshipStat_" + str(fulllistofyears[i])
    UnprocessedDF[NewColumnName] = UnprocessedDF.apply(lambda row: createfinalmarr(row, a, b,c), axis=1)

ListOfColumnstoKeep = ['EMP_Hours','KEY!AGEDOL','KEY!SEX','FinalRelationshipStat_','EducationStat','HasSibling','SingleorDoubleParent','EMP_HOURS']

columns_to_keep = [col for col in UnprocessedDF.columns if any(value in col for value in ListOfColumnstoKeep)]


UnprocessedDF = UnprocessedDF[columns_to_keep]

In [187]:
#Shift Age Up by One Year since we start in 1997 not 1996

UnprocessedDF['KEY!AGEDOL'] = UnprocessedDF['KEY!AGEDOL'] + 1

years = [1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]


In [188]:
import re

years = [1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

# Calculate birth year for each individual
UnprocessedDF['BirthYear'] = min(years) - UnprocessedDF['KEY!AGEDOL']

# Identify all base column names (e.g., 'FinalState', 'OtherState', etc.)
base_names = set([col.rsplit('_', 1)[0] for col in UnprocessedDF.columns if col.split('_')[-1].isdigit()])

UnprocessedDF = UnprocessedDF[sorted(UnprocessedDF.columns)]

In [189]:
# Create age-based columns
for base_name in base_names:
    # Calculate the age range across all years
    min_age = (min(years) - UnprocessedDF['BirthYear']).min()
    max_age = (max(years) - UnprocessedDF['BirthYear']).max()
    
    for age in range(int(min_age), int(max_age) + 1):
        new_col_name = f"{base_name}_{age}"
        UnprocessedDF[new_col_name] = None

        for year in years:
            year_col = f"{base_name}_{year}"
            if year_col in UnprocessedDF.columns:
                # Calculate the age for each individual in this specific year
                current_age = year - UnprocessedDF['BirthYear']
                mask = (current_age == age)
                UnprocessedDF.loc[mask, new_col_name] = UnprocessedDF.loc[mask, year_col]

In [190]:
year_columns = [col for col in UnprocessedDF.columns if col.split('_')[-1] in map(str, years)]
UnprocessedDF = UnprocessedDF.drop(year_columns, axis=1)


suffixes = ['_13', '_14', '_15', '_16', '_36', '_37', '_38', '_39']

# Get a list of columns that end with any of these suffixes
columns_to_drop = [col for col in UnprocessedDF.columns if any(col.endswith(suffix) for suffix in suffixes)]


ProcessedDF = UnprocessedDF.drop(columns=columns_to_drop)

In [191]:
ProcessedDF = ProcessedDF.rename(columns={'KEY!AGEDOL': 'Age', 'KEY!SEX': 'Sex'})

ProcessedDF['Sex'] = ProcessedDF['Sex'].replace(1, 0)
ProcessedDF['Sex'] = ProcessedDF['Sex'].replace(2, 1)

ProcessedDF.to_csv('ProcessedDF-New.csv')

In [ ]:
end = 3551
start = 1

sample_size = int(0.2 * (end - start + 1))
# Generate the random sample
np.random.seed(42) 
sampleid = np.random.choice(range(start, end+1), size=sample_size, replace=False)
sampleid.sort()
sampleidcsv = pd.DataFrame(sampleid)
sampleidcsv.to_csv('ProperSample.csv')